### **Data Mining Using SQL**

<font color="red">File access required:</font> In Colab this notebook requires first uploading files **Shop.csv** and **Movies.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure these files are in the same workspace as the notebook.

In [ ]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load tables from CSV files
# Shop
with open('/content/drive/MyDrive/Google Colab/Data Mining Using Python and SQL/Copy of Shop.csv') as f: Shop = pd.read_csv(f, index_col=0)
%sql drop table if exists Shop;
%sql --persist Shop
# Movies
with open('/content/drive/MyDrive/Google Colab/Data Mining Using Python and SQL/Copy of Movies.csv') as f: Movies = pd.read_csv(f, index_col=0)
%sql drop table if exists Movies;
%sql --persist Movies

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted movies'

## Shopping dataset - frequent item-sets

In [ ]:
%%sql
select * from Shop

 * sqlite://
Done.


TID,item
1,milk
1,eggs
1,juice
2,milk
2,juice
2,cookies
3,eggs
3,chips
4,milk
4,eggs


### Frequent item-sets of two

#### All pairs of items occurring together

In [ ]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item

 * sqlite://
Done.


item1,item2
eggs,milk
eggs,juice
juice,milk
juice,milk
cookies,milk
cookies,juice
chips,eggs
eggs,milk
juice,milk
cookies,milk


#### Now with number of transactions they occur together in

In [ ]:
%%sql
select T1.item as item1, T2.item as item2, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item

 * sqlite://
Done.


item1,item2,count()
chips,cookies,1
chips,eggs,1
chips,juice,1
chips,milk,1
cookies,juice,2
cookies,milk,2
eggs,juice,1
eggs,milk,2
juice,milk,3


#### Total number of transactions

In [ ]:
%%sql
select count(distinct TID) from Shop

 * sqlite://
Done.


count(distinct TID)
5


#### Put it together for frequent item-sets

In [ ]:
support = .3
print(support)

0.3


In [ ]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2
cookies,juice
cookies,milk
eggs,milk
juice,milk


### Frequent item-sets of three

#### All triples of items occurring together

In [ ]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item

 * sqlite://
Done.


item1,item2,item3
eggs,juice,milk
cookies,juice,milk
cookies,juice,milk
chips,juice,milk
chips,cookies,milk
chips,cookies,juice


#### Now with number of transactions they occur together in

In [ ]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3, count()
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


item1,item2,item3,count()
chips,cookies,juice,1
chips,cookies,milk,1
chips,juice,milk,1
cookies,juice,milk,2
eggs,juice,milk,1


#### Put it together for frequent item-sets

In [ ]:
support = .3
print(support)

0.3


In [ ]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2,item3
cookies,juice,milk


### <font color = 'green'>**Your Turn - Movies dataset frequent item-sets**</font>

In [ ]:
%%sql
select * from Movies limit 15

 * sqlite://
Done.


TID,item
145755,The Fault in Our Stars
145755,Boyhood
145755,Big Hero 6
145755,The Imitation Game
145755,Inside Out
117070,Inside Out
239040,Inside Out
177796,Gone Girl
177796,Magic Mike XXL
126875,The Imitation Game


In [ ]:
%%sql
select count(distinct TID) as numUsers,
       count(distinct item) as numMovies
from Movies

 * sqlite://
Done.


numUsers,numMovies
1382,123


#### Mine for frequent item-sets of three and four items in the Movies dataset. Find a single support threshold where the number of frequent item-sets of three items is more than 10 but less than 20, and the number of frequent item-sets of four items is more than 0.

In [ ]:
support = .03
print(support)

0.03


In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
# Frequent item-sets of three
%%sql
for i1 in Mitems:
    for i2 in Mitems:
        for i3 in Mitems:
            if i1 < i2 and i2 < i3:
                common = len(Mitems[i1] & Mitems[i2] & Mitems[i3])
                if common/Mnumtrans > support:
                    print(i1, '|', i2, '|', i3)

 * sqlite://
(sqlite3.OperationalError) near "for": syntax error
[SQL: for i1 in Mitems:
    for i2 in Mitems:
        for i3 in Mitems:
            if i1 < i2 and i2 < i3:
                common = len(Mitems[i1] & Mitems[i2] & Mitems[i3])
                if common/Mnumtrans > support:
                    print(i1, '|', i2, '|', i3)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
# Frequent item-sets of four
%%sql
for i1 in Mitems:
    for i2 in Mitems:
        for i3 in Mitems:
            for i4 in Mitems:
                if i1 < i2 and i2 < i3 and i3 < i4:
                    common = len(Mitems[i1] & Mitems[i2] & Mitems[i3] & Mitems[i4])
                    if common/Mnumtrans > support:
                        print(i1, '|', i2, '|', i3, '|', i4)

 * sqlite://
(sqlite3.OperationalError) near "for": syntax error
[SQL: for i1 in Mitems:
    for i2 in Mitems:
        for i3 in Mitems:
            for i4 in Mitems:
                if i1 < i2 and i2 < i3 and i3 < i4:
                    common = len(Mitems[i1] & Mitems[i2] & Mitems[i3] & Mitems[i4])
                    if common/Mnumtrans > support:
                        print(i1, '|', i2, '|', i3, '|', i4)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Shopping dataset - association rules

### Association rules with one item on the left-hand side

In [ ]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules; include number of transactions

In [ ]:
%%sql
select item, count() as numtrans
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item,numtrans
chips,2
cookies,2
eggs,3
juice,3
milk,4


#### Do the same query but save the result in a table called "Frequents"

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
5 rows affected.
Done.


item,numtrans
chips,2
cookies,2
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,numtrans,count()
chips,cookies,2,1
chips,eggs,2,1
chips,juice,2,1
chips,milk,2,1
cookies,chips,2,1
cookies,juice,2,2
cookies,milk,2,2
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2


#### Add check for specified confidence to get final result

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
(sqlite3.OperationalError) no such table: Frequent
[SQL: select T1.item as LHS, T2.item as RHS
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item
having 1.0*count() / F.numtrans > ?]
[parameters: (0.92,)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Association rules with two items on the left-hand side

In [ ]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for specified confidence to get final result

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS1,LHS2,RHS
juice,milk,cookies


## Shopping dataset - association rules with lift instead of confidence

#### Compute overall frequency of each item -- needed for lift calculation

In [ ]:
%%sql
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item

 * sqlite://
Done.


item,1.0*count() / (select count(distinct TID) from Shop)
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


#### Do the same query but save the result in a table called "ItemFreq"

In [ ]:
%%sql
drop table if exists ItemFreq;
create table ItemFreq(item, freq);
insert into ItemFreq
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item;
select * from ItemFreq

 * sqlite://
Done.
Done.
5 rows affected.
Done.


item,freq
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


### Association rules with one item on the left-hand side

In [ ]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions.

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,numtrans,count()
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2
juice,chips,3,1
juice,cookies,3,2
juice,eggs,3,1
juice,milk,3,3
milk,chips,4,1
milk,cookies,4,2
milk,eggs,4,2


#### Add check for lift > 1 to get final result

In [ ]:
%%sql
select T1.item as LHS, T2.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Frequent F, ItemFreq I
where T1.item = F.item
and T1.TID = T2.TID
and T2.item = I.item
and T1.item != T2.item
group by T1.item, T2.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS,RHS,lift
juice,cookies,1.6666666666666665
juice,milk,1.25
milk,cookies,1.25
milk,juice,1.25


### Association rules with two items on the left-hand side

In [ ]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [ ]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for lift > 1 to get final result

In [ ]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Shop T3, Frequent F, ItemFreq I
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID and T3.item = I.item
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS1,LHS2,RHS,lift
juice,milk,cookies,1.6666666666666665


### <font color = 'green'>**Your Turn - Movies dataset association rules**</font>

#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and confidence thresholds (need not be the same) so the number of association rules is more than 10 but less than 20.


In [85]:
support = .01
confidence = .92
print(support, confidence)

0.01 0.92


In [86]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, count()
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support;
select * from Frequent limit 10; -- Displaying the first 10 frequent itemsets

 * sqlite://
Done.
Done.
81 rows affected.
Done.


item1,item2,item3,numtrans
Big Eyes,Boyhood,Gone Girl,14
Big Eyes,Boyhood,The Imitation Game,15
Big Eyes,Gone Girl,The Imitation Game,17
Big Hero 6,Boyhood,Fury,29
Big Hero 6,Boyhood,Gone Girl,56
Big Hero 6,Boyhood,Inside Out,39
Big Hero 6,Boyhood,The Fault in Our Stars,23
Big Hero 6,Boyhood,The Imitation Game,57
Big Hero 6,Boyhood,Wild Tales,14
Big Hero 6,Calvary,Gone Girl,15


In [87]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as LHS3, T4.item as RHS
from Movies T1, Movies T2, Movies T3, Movies T4, Frequent F
where T1.item = F.item1 and T2.item = F.item2 and T3.item = F.item3
and T1.TID = T2.TID and T2.TID = T3.TID and T3.TID = T4.TID
and T4.item != T1.item and T4.item != T2.item and T4.item != T3.item
group by T1.item, T2.item, T3.item, T4.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS1,LHS2,LHS3,RHS
Big Hero 6,Boyhood,Fury,Gone Girl
Big Hero 6,Boyhood,Fury,The Imitation Game
Big Hero 6,Boyhood,Wild Tales,Gone Girl
Big Hero 6,Boyhood,Wild Tales,The Imitation Game
Big Hero 6,Calvary,Gone Girl,The Imitation Game
Big Hero 6,Calvary,The Imitation Game,Gone Girl
Big Hero 6,Fury,Transformers: Age of Extinction,The Imitation Game
Big Hero 6,The Hunger Games: Mockingjay - Part 2,The Imitation Game,Inside Out
Boyhood,Calvary,Fury,Gone Girl
Boyhood,Calvary,Fury,The Imitation Game


#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and lift thresholds so the number of association rules is more than 10 but less than 20.


In [88]:
support = .08
lift = .5
print(support, lift)

0.08 0.5


In [73]:
%%sql
drop table if exists ItemFreq;

create table ItemFreq(item, freq);
insert into ItemFreq
select item, 1.0*count() / (select count(distinct TID) from Movies)
from Movies
group by item;
select * from ItemFreq limit 10; -- Displaying the first 10 item frequencies

 * sqlite://
Done.
Done.
123 rows affected.
Done.


item,freq
400 Days,0.002170767004341534
A Walk in the Woods,0.000723589001447178
About Alex,0.005065123010130246
Action Jackson,0.000723589001447178
American Ultra,0.013024602026049204
Annie,0.004341534008683068
Anti-Social,0.000723589001447178
Appropriate Behaviour,0.001447178002894356
Ascension,0.002170767004341534
Aziz Ansari: Live at Madison Square Garden,0.000723589001447178


In [89]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, item3, numtrans);
insert into Frequent
select T1.item, T2.item, T3.item, count()
from Movies T1, Movies T2, Movies T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Movies) > :support;
select * from Frequent limit 10; -- Displaying the first 10 frequent itemsets

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,item3,numtrans
Big Hero 6,Gone Girl,The Imitation Game,119


In [90]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as LHS3, T4.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Movies T1, Movies T2, Movies T3, Movies T4, Frequent F, ItemFreq I
where T1.item = F.item1 and T2.item = F.item2 and T3.item = F.item3
and T1.TID = T2.TID and T2.TID = T3.TID and T3.TID = T4.TID
and T4.item = I.item -- Join with ItemFreq to get the frequency of the RHS item
and T4.item != T1.item and T4.item != T2.item and T4.item != T3.item
group by T1.item, T2.item, T3.item, T4.item
having (1.0*count() / F.numtrans) / I.freq > :lift

 * sqlite://
Done.


LHS1,LHS2,LHS3,RHS,lift
Big Hero 6,Gone Girl,The Imitation Game,About Alex,1.65906362545018
Big Hero 6,Gone Girl,The Imitation Game,Action Jackson,11.61344537815126
Big Hero 6,Gone Girl,The Imitation Game,American Ultra,3.8711484593837535
Big Hero 6,Gone Girl,The Imitation Game,Annie,3.871148459383753
Big Hero 6,Gone Girl,The Imitation Game,Ascension,3.871148459383753
Big Hero 6,Gone Girl,The Imitation Game,Best of Enemies,3.871148459383753
Big Hero 6,Gone Girl,The Imitation Game,Big Eyes,4.0046363372935385
Big Hero 6,Gone Girl,The Imitation Game,Boyhood,2.343063541205956
Big Hero 6,Gone Girl,The Imitation Game,Breathe,11.61344537815126
Big Hero 6,Gone Girl,The Imitation Game,Calvary,4.516339869281046
